In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

d:\python\envs\py361\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\python\envs\py361\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\python\envs\py361\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\python\envs\py361\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or 

处理训练集标签

In [2]:
import csv
 
path = 'train.txt'
# newline这里是去除行之间的空行

df = pd.read_csv(path, sep=',', header=None,skiprows=1)
df.columns = ['guid','tag']
column_dict = {"negative": -1,"neutral": 0,"positive":1}
    # new_df = df.replace({"col1": column_dict})
df['tag'] = df['tag'].map(column_dict)
df = df.sort_values(by="guid")
#df
df.to_csv('data.csv',index=False)

处理图文数据

加载处理好的向量化的图文数据矩阵，并调整张量大小

In [3]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=np.inf)
a=np.load('picture_feature_train.npy',allow_pickle=True)
#print(a)
print(a.shape)

(4000, 1, 512)


In [2]:
#导入写好的注意力机制
np.set_printoptions(threshold=np.inf)
attention_keras = __import__("keras_attention_mechanism_model")

text_data = np.load('normalized_data.npy')
#print(text_data.shape)
text_data=text_data[:4000,:1500]
#print(text_data.shape)
text_data = text_data.reshape([-1, 30, 50])
picture_filename = 'picture_feature_train.npy'
picture_data = np.load(picture_filename)
picture_data=picture_data.reshape(4000,16,32)

with open('data.csv', 'r') as f:
    df = pd.read_csv(f, delimiter=',', header=None)
#print(df)
labels=df[1][1:4000].values.astype(float)
labels = np.array(labels)
#print(labels)
from collections import Counter
d = Counter(labels)
d_s = sorted(d.items(),key=lambda x:x[1],reverse=True)
print(d_s)

[(1.0, 2387), (-1.0, 1193), (0.0, 419)]


参数设置

In [3]:
#参数
batch_size = 16
epochs=50

开始训练

In [4]:
np.random.seed(5)
index = [i for i in range(len(labels))]
index = np.random.permutation(index)
text_data = text_data[index]
picture_data = picture_data[index]
labels = labels[index]
labels = tf.keras.utils.to_categorical(labels)

text_train_data = text_data[:3500]
text_test_data = text_data[3500:]
picture_train_data = picture_data[:3500]
picture_test_data = picture_data[3500:]
train_labels = labels[:3500]
test_labels = labels[3500:]

#train_labels[train_labels == 0] = 2
#test_labels[test_labels == 0] = 2

# 添加一个新的类别 0，用于表示第三个分类
# 将原始标签中的 1 替换为 0
#train_labels[train_labels == 1] = 0
#test_labels[test_labels == 1] = 0

# 将原始标签中的 2 替换为 1
#train_labels[train_labels == 2] = 1
#test_labels[test_labels == 2] = 1
train_labels=np.argmax(train_labels,axis=1)
test_labels=np.argmax(test_labels,axis=1)

#转换为三种类别的情绪向量标签以计算损失函数
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=3)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=3)


#根据图文张量大小设置模型输入输出函数
S_inputs1 = tf.keras.Input(shape=(30,50), dtype='float32')
S_inputs2 = tf.keras.Input(shape=(16,32), dtype='float32')

#位置信息编码
text_embeddings = attention_keras.Position_Embedding()(S_inputs1)
picture_embeddings = attention_keras.Position_Embedding()(S_inputs2)

#embedding传入图文数据
#多头注意力机制
O_seq_1 = attention_keras.Attention(8, 16)([picture_embeddings, text_embeddings, text_embeddings])
O_seq_1 = tf.keras.layers.Activation('relu')(O_seq_1)

O_seq_2 = attention_keras.Attention(8, 16)([text_embeddings, picture_embeddings, picture_embeddings])
O_seq_2 = tf.keras.layers.Activation('relu')(O_seq_2)

O_seq_3 = attention_keras.Attention(8, 16)([picture_embeddings, picture_embeddings, picture_embeddings])
O_seq_3 = tf.keras.layers.Activation('relu')(O_seq_3)
#拼接三种张量
O_seq = tf.concat((O_seq_1, O_seq_2, O_seq_3), axis=1)
#O_seq = tf.concat((O_seq_1), axis=1)
#平均池化
O_seq = tf.keras.layers.GlobalAveragePooling1D()(O_seq)
#正则化防止过拟合
O_seq = tf.keras.layers.Dropout(0.3)(O_seq)
#增加全连接层
outputs = tf.keras.layers.Dense(3, activation='softmax')(O_seq)
#print(outputs)
#根据输入输出的张量大小实体化一个网络模型
model = tf.keras.models.Model(inputs=[S_inputs1, S_inputs2], outputs=outputs)

#优化器
adam = tf.keras.optimizers.Adam(lr=5e-4)
#adam = tf.keras.optimizers.Adam(lr=2e-4)#85
#配置模型
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#开始训练
print('Train...')
#filepath="model_save/a.h5"
#checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=filepath,monitor='val_accuracy',verbose=1,save_best_only=True,)
#checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=filepath,verbose=1,save_best_only=True,)
#model.fit(x={'input_1': text_train_data, 'input_2': picture_train_data}, y=train_labels, batch_size=batch_size, epochs=50, verbose=2,
#         validation_data=([text_test_data, picture_test_data], test_labels))
model.fit(x={'input_1': text_train_data, 'input_2': picture_train_data}, y=train_labels, batch_size=batch_size, epochs=epochs, verbose=2,
         validation_data=([text_test_data, picture_test_data], test_labels))
# Fit the model
#model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=30, batch_size=200,callbacks=[checkpoint],verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train...
Train on 3500 samples, validate on 499 samples
Epoch 1/50
3500/3500 - 3s - loss: 0.3890 - acc: 0.8777 - val_loss: 0.4173 - val_acc: 0.8597
Epoch 2/50
3500/3500 - 3s - loss: 0.3291 - acc: 0.9003 - val_loss: 0.4104 - val_acc: 0.8597
Epoch 3/50
3500/3500 - 3s - loss: 0.3319 - acc: 0.9003 - val_loss: 0.4074 - val_acc: 0.8597
Epoch 4/50
3500/3500 - 2s - loss: 0.3291 - acc: 0.9003 - val_loss: 0.4076 - val_acc: 0.8597
Epoch 5/50
3500/3500 - 2s - loss: 0.3284 - acc: 0.9003 - val_loss: 0.4242 - val_acc: 0.8597
Epoch 6/50
3500/3500 - 3s - loss: 0.3318 - acc: 0.9003 - val_loss: 0.4187 - val_acc: 0.8597
Epoch 7/50
3500/3500 - 3s - loss: 0.3271 - acc: 0.9003 - val_loss: 0.4086 - val_acc: 0.8597
Epoch 8/50
3500/3500 - 2s - loss: 0.3272 - acc: 0.9003 - val_loss: 0.4344 - val_acc: 0.8597
Epoch 9/50
3500/3500 - 2s - loss: 0.3246 - acc: 0.9003 - val_loss: 0.4095 - val_acc: 0.859

In [7]:

# 加载要预测的数据
text_filename = 'normalized_data.npy'
text_data = np.load(text_filename)
text_data=text_data[4000:4511,:1500]
text_data = text_data.reshape([-1, 30, 50])
visual_filename = 'picture_feature_test.npy'
visual_data = np.load(visual_filename)
visual_data=visual_data.reshape(511,16,32)

# 进行预测
predictions = model.predict({'input_1': text_data, 'input_2': visual_data})
predictions=np.argmax(predictions,axis=1)


In [21]:
paths = 'test_without_label.txt'
df1 = pd.read_csv(paths, sep=',', header=None,skiprows=1)
df1=df1.drop(columns=1)
df1['tag'] = predictions
df1.columns = ['guid','tag']
column_dict = {-1: "negative",0: "neutral",1:"positive"}
df1['tag'] = df1['tag'].map(column_dict)
print(df1)
df1.to_csv('submit_result.csv',sep=',',index=False)

     guid       tag
0       8  positive
1    1576  positive
2    2320  positive
3    4912  positive
4    3821  positive
..    ...       ...
506  1048   neutral
507  1059  positive
508  1485  positive
509  3195   neutral
510  2029  positive

[511 rows x 2 columns]
